In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
import ast
from itertools import chain

In [ ]:
# (helps with displaying dataframes containing long strings)
pd.set_option('display.max_colwidth', 0)

# Load data

In [ ]:
# Get mapping of ICD-9 codes to symptoms (i.e., based on ChatGPT responses)
temp_fp = "icd9_symptom_map_v2.csv"
icd9_symptoms_map = pd.read_csv(temp_fp, dtype={"icd9_first_3": str, "symptoms": str})

In [ ]:
# Some ChatGPT responses included some extra preamble and formatting - remove this for ease of analysis
icd9_symptoms_map["symptoms_clean"] =  icd9_symptoms_map["symptoms"].apply(lambda x: x.split("Symptoms: ")[1])

In [ ]:
# Convert symptoms from string to list
icd9_symptoms_map["symptoms_list"] = icd9_symptoms_map["symptoms_clean"].apply(lambda x: [y.lower() for y in x.split(", ")])

In [ ]:
# Get symptoms extracted from clinical notes for each admission
temp_fp = "notes_and_symptoms.csv"
notes_and_symptoms = pd.read_csv(temp_fp)

In [ ]:
# If there are multiple records with the same HADM_ID, just keep the first one
notes_and_symptoms = notes_and_symptoms.drop_duplicates("HADM_ID", keep="first")

In [ ]:
# Convert symptoms from string to list
notes_and_symptoms["symptoms"] = notes_and_symptoms["symptoms"].apply(lambda x: ast.literal_eval(x))

In [ ]:
# Get unique symptoms for each admission
notes_and_symptoms["symptoms_unique"] = notes_and_symptoms["symptoms"].apply(lambda x: list(set(x)))

In [ ]:
# Get diagnoses
diagnoses = pd.read_csv("DIAGNOSES_ICD.csv.gz")

In [ ]:
# For simplicity, pick first diagnosis in sequence for each admission
diagnoses = diagnoses.query("SEQ_NUM == 1")
diagnoses = diagnoses[["HADM_ID", "ICD9_CODE"]]

In [ ]:
# Get first 3 digits of ICD9 code
diagnoses["icd9_first_3"] = diagnoses["ICD9_CODE"].apply(lambda x: x[0:3])

In [ ]:
# Merge diagnoses with symptoms from discharge notes
merge_df = pd.merge(diagnoses, notes_and_symptoms, on="HADM_ID", how="inner")

In [ ]:
# Filter ICD9-symptoms map to only include diagnoses that appear in the data (first 3 digits only)
icd9_symptoms_map = icd9_symptoms_map[icd9_symptoms_map["icd9_first_3"].isin(merge_df["icd9_first_3"])]

In [ ]:
# Filter merged data to only include diagnoses where we looked up associated symptoms
# (i.e., excluding supplementary info)
merge_df = merge_df[merge_df["icd9_first_3"].isin(icd9_symptoms_map["icd9_first_3"])]

In [ ]:
# TEMP
train = merge_df.sample(100)

# Build graph

In [ ]:
G = nx.Graph()

In [ ]:
all_icd = icd9_symptoms_map["icd9_first_3"].values
all_symptoms = list(
    set(chain.from_iterable(icd9_symptoms_map["symptoms_list"])).union(
        set(chain.from_iterable(train["symptoms_unique"])
    )
))
all_admissions = train["HADM_ID"].values

In [ ]:
# Add nodes
G.add_nodes_from(["icd_" + x for x in all_icd])
G.add_nodes_from(["symptom_" + x for x in all_symptoms])
G.add_nodes_from(["hadm_" + str(x) for x in all_admissions])

In [ ]:
# Create a mapping from node labels to indices
node_mapping = {node: idx for idx, node in enumerate(G.nodes())}

In [ ]:
# Add edges - ICD9 to symptom
for i in range(0, icd9_symptoms_map.shape[0]):
    temp_icd = icd9_symptoms_map["icd9_first_3"].iloc[i]
    temp_symptoms_list = icd9_symptoms_map["symptoms_list"].iloc[i]
    for s in temp_symptoms_list:
        G.add_edges_from([("icd_" + temp_icd, "symptom_" + s)])

In [ ]:
# Add edges - admission to symptom
for i in range(0, train.shape[0]):
    temp_admit = train["HADM_ID"].iloc[i]
    temp_symptoms_list = train["symptoms"].iloc[i]
    for s in temp_symptoms_list:
        G.add_edges_from([("hadm_" + str(temp_admit), "symptom_" + s)])

In [ ]:
# Convert to PyTorch Geometric data
edge_index = torch.tensor([[node_mapping[edge[0]], node_mapping[edge[1]]] for edge in G.edges()]).t().contiguous()

# Use identity matrix as node features
x = torch.eye(len(G.nodes()))

PyG_data = Data(x=x, edge_index=edge_index)

In [ ]:
# Get node indices for ICD9 codes
icd9_node_idx = torch.tensor([node_mapping[node] for node in ["icd_" + x for x in all_icd]])

In [ ]:
# Get node indices for admissions
hadm_node_idx = torch.tensor([node_mapping[node] for node in ["hadm_" + str(x) for x in all_admissions]])

In [ ]:
# Create label vector for each admission
# (i.e., 1 in the column corresponding with the primary diagnosis, 0 elsewhere)
mlb = MultiLabelBinarizer(classes=all_icd)
train_labels = mlb.fit_transform(train["icd9_first_3"].apply(lambda x: [x]))
train_labels = torch.tensor(train_labels, dtype=torch.float)

# Train GNN model

In [ ]:
# Create simple GNN model with two graph convolutional layers
class GNN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x, edge_index)
        return torch.nn.functional.log_softmax(x, dim=1)

In [ ]:
# Initialize model
model = GNN(num_features=PyG_data.num_features, hidden_dim=16, num_classes=len(all_diags))

In [ ]:
# Set optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Train model
model.train()
for i in range(0, 200):
    optimizer.zero_grad()
    full_output = model(PyG_data)
    
    # Get output for ICD9 nodes
    icd9_output = full_output[icd9_node_idx]
    
    # Get output for admission nodes
    hadm_output = full_output[hadm_node_idx]
    
    loss = criterion(hadm_output, train_labels)
    print(loss)
    loss.backward()
    optimizer.step()

# Evaluate GNN model

In [ ]:
# Evaluate training accuracy
pred_idx = hadm_output.argmax(axis=1)
gold_idx = train_labels.argmax(axis=1)
train_accuracy = pred_idx.eq(gold_idx).sum().item() / len(gold_idx)
print(train_accuracy)